In [1]:
import pandas as pd

vaccine_file = "Folkhalsomyndigheten_Covid19_Vaccine.xlsx"

veckodata_vaccin_df = pd.read_excel(vaccine_file, sheet_name = "Vaccinerade kommun och ålder")

In [2]:
counties = veckodata_vaccin_df["Län_namn"].unique().__len__()

print(f"Det finns {counties} län representerade i datasetet.")

Det finns 21 län representerade i datasetet.


In [3]:
municipalities = veckodata_vaccin_df["Kommun_namn"].unique().__len__()

print(f"Det finns {municipalities} kommuner representerade i datasetet.")

Det finns 290 kommuner representerade i datasetet.


In [4]:
population = veckodata_vaccin_df["Befolkning"].sum()

print(f"Det finns {population} individer representerade i datasetet.")

Det finns 8541497 individer representerade i datasetet.


In [5]:
swedish_population = 10427296

population_under_16 = swedish_population - population

print(f"Det finns {population_under_16} personer under 16 år i Sverige.")

Det finns 1885799 personer under 16 år i Sverige.


In [6]:
counties_sum = veckodata_vaccin_df.groupby(["Län_namn"])[["Antal minst 1 dos", "Antal färdigvaccinerade", "Befolkning"]].sum().reset_index()

counties_sum["Andel minst 1 dos"] = counties_sum["Antal minst 1 dos"] / counties_sum["Befolkning"]
counties_sum["Andel färdigvaccinerade"] = counties_sum["Antal färdigvaccinerade"] / counties_sum["Befolkning"]

In [7]:
import plotly_express as px

fig = px.bar(
    counties_sum.sort_values(by = "Andel färdigvaccinerade", ascending = False), 
    x = "Län_namn", 
    y = ["Andel minst 1 dos", "Andel färdigvaccinerade"], 
    barmode = "group", 
    title = "Andel vaccinerade, per län",
    labels = {"variable": "Variabel"}
)

fig.update_xaxes(title_text = "Län")
fig.update_yaxes(title_text = "Andel")
fig.update_traces(hovertemplate = "%{y:.2%}")
fig.update_layout(yaxis = { "tickformat": ",.2%"})

fig.write_html("./Visualiseringar/andel_vaccinerade.html")

In [8]:
fig = px.bar(
    counties_sum, 
    x = "Län_namn", 
    y = ["Antal minst 1 dos", "Antal färdigvaccinerade", "Befolkning"], 
    barmode = "group", 
    title = "Vaccination och befolkning över 16 år, per län",
    labels = {"variable": "Variabel"}
)

fig.update_xaxes(title_text = "Län")
fig.update_yaxes(title_text = "Antal")
fig.update_traces(hovertemplate = "%{y:.0f}")

fig.write_html("./Visualiseringar/antal_vaccinerade.html")